### Упражнение

1. Оптимизировать загрузку памяти датафреймом статистики преступлений в Лос-Анджелесе.
- Столбец 'Date Reported' должн быть представлен типом данных  datetime64[ns]
2. Добавить столбец полного времени совершенного преступления (тип данных datetime64[ns])
3. Добавить столбец дня недели совершенного преступления (тип данных category)

### Упражнение по визуализации

1. Стилизировать столбец таблицы общего количества преступлений по дням недели.    


2. Построить распределение количества преступлений по дням недедли в зависимости от района:
- линейна диаграмма;
- гистограмма по дням;
- столбчатая диаграмма накопления ( stacked bar chart); 
- площадная диаграмма (стыковачная и нестыковочная;
- диаграмма рассеяния количества преступлений по дням недели (два графика scatter_matrix и pairplot);
- распределение количества преступлений в любой день недели (например:пятница).

3. Анализ количества преступлений по районам:
- гистограмма распределения количества преступлений в зависимости от района;
- распределение количества преступлений за любой год (например: 2010 год);
- показать через jointplot динамику изменений количества преступлений в сравнении 2010 и 2017 года;
- годовая динамика изменения количества преступлений по районам (линейный график  с легендами и удобным масштабом).

4. Анализ преступлений в заданном районе:
- количество отдельно взятого вида преступления распределенного по дням недели;
- построить таблицу (вид преступления по дням недели) и подсветить максимальные и минимальные значения для каждого вида преступления разным цветом;
- сравнение распределения 20 наиболее часто встречающихся преступлений по дня недели для заданного района (показать через boxplot);
- проанализировать коэфициент корреляции вида преступления и дня недели (результат в виде heatmap на которой отображено только корреляция вида преступления от дня недели)
- линейный график коэффициент корреляции дня недели и вида преступления в абсолютных значениях.

5. Сравнение количества преступлений по дням недели и районам:
- построить линейный график для трех любых районов;
- построить линейный график для всех районов сразу.

6. Доля преступлений в конкретном районе от общего количества:
- Bar график для всех районов по всей статистики;
- Bar график для всех районов по годам.

7. box график статистики всех преступлений в распределении по годам.   


8. Интерактивный график:
- количество преступлений от дня недели для отдельного района;
- количество преступлений от района для от дельного дня недели.

In [ ]:
# отключаем предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

import datetime

# отображать графики прямо в jupyter'e
%pylab inline
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'retina' # ’retina’ (svg) - для еще более четкого отображения

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 6,3
import seaborn as sns

# задаем некоторые настройки pandas, регулирующие
# формат вывода
# pd.set_option('display.notebook_repr_html', False)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.width', None)

# импортируем библиотеку matplotlib для построения графиков
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.plotting import scatter_matrix

# задаем стартовое значение для генератора
# случайных чисел
seedval = 111111

from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [ ]:
# создание словаря, в котором ключи - имена столбцов, 
# а значения - типы столбцов
column_types = {   'DR Number': 'int64',
    'Time Occurred': 'int64',
    'Area ID': 'int64',
    'Area Name': 'category',
    'Reporting District': 'int64',           
    'Crime Code': 'int64',
    'Crime Code Description': 'category',
    'MO Codes': 'category',
    'Victim Age': 'float32',
    'Victim Sex': 'category',
    'Victim Descent': 'category',
    'Premise Code': 'float32',
    'Premise Description': 'category',
    'Weapon Used Code': 'float32',
    'Weapon Description': 'category',
    'Status Code': 'category',
    'Status Description': 'category',
    'Crime Code 1': 'float32',
    'Crime Code 2': 'float32',
    'Crime Code 3': 'float32',
    'Crime Code 4': 'float32',
    'Address': 'category',
    'Cross Street': 'category',
    'Location ': 'category'}

In [ ]:
# считывание данных с нужными типами из словаря, преобразование даты
crime_data = pd.read_csv('Crime_Data_from_2010.csv', 
                         dtype = column_types,
                         parse_dates = ['Date Occurred', 'Date Reported'],
                         infer_datetime_format = True)
crime_data.info(memory_usage='deep')

In [ ]:
# создание NumPy массива и обработка его в цикле
# для добавления нулей ко времени, в котором содержится только один символ
Time_Occurred_NP = np.array(crime_data['Time Occurred'], dtype = np.str)
for i in range(0, len(Time_Occurred_NP)):
    if len(Time_Occurred_NP[i]) <= 1:
        Time_Occurred_NP[i] = Time_Occurred_NP[i] + '0'
crime_data['Time Occurred'] = Time_Occurred_NP     

In [ ]:
# создание столбца полного времени совершенного преступления 
crime_data['Date_Occurred_Full'] = crime_data['Date Occurred'].astype(str) + ' ' + crime_data['Time Occurred']

In [ ]:
# представление полного времени совершенного преступления в читаемом виде
crime_data['Date_Occurred_Full'] = pd.to_datetime(crime_data['Date_Occurred_Full'].str.strip(),
                                                  format='%Y-%m-%d %H%M')

In [ ]:
# добавлениe столбца дня недели совершённого преступления
# изменение типа на category
crime_data['Weekday_Occurred'] = crime_data['Date_Occurred_Full'].dt.strftime('%A').astype('category')

In [ ]:
# создание датафрейма с количеством престплений по дням недели
weekday_occ_count = pd.DataFrame(crime_data['Weekday_Occurred'].value_counts().values,
                                 index = crime_data['Weekday_Occurred'].value_counts().index, 
                                 columns = ['Crime_Occured'])

In [ ]:
#  стилизация столбца
weekday_occ_count.style.bar(color = '#f09080')

In [ ]:
# создание сводной таблицы из датафрейма
# для вывода количетсва преступлений по дням недели и районам
area_weekday_plot = crime_data.pivot_table(
                        index = 'Area Name', 
                        columns = 'Weekday_Occurred', 
                        values = 'Date_Occurred_Full', 
                        aggfunc = 'count')

area_weekday_plot.head()

In [ ]:
# линейна диаграмма распределения количества преступлений по дням недедли в зависимости от района
area_weekday_plot.plot(grid = True, figsize = (10, 5), colormap = 'Set2');

In [ ]:
# гистограмма по дням распределения количества преступлений по дням недедли в зависимости от района
area_weekday_plot.hist(figsize = (12,8), color = '#78abff');

In [ ]:
# столбчатая диаграмма накопления распределения количества преступлений по дням недедли в зависимости от района
area_weekday_plot.plot(kind='bar', stacked=True, figsize = (15,7), rot = 45, colormap = 'Pastel2');

In [ ]:
# площадная стыковачная диаграмма распределения количества преступлений по дням недедли в зависимости от района
area_weekday_plot.plot(kind='area', figsize = (11,7), colormap = 'Pastel1');

In [ ]:
#  площадная нестыковачная диаграмма накопления распределения количества преступлений по дням недедли в зависимости от района
area_weekday_plot.plot(kind='area', stacked = False, figsize = (11, 7), colormap = 'Accent');

In [ ]:
# диаграмма рассеяния количества преступлений по дням недели (pairplot)
sns.pairplot(area_weekday_plot, plot_kws=dict(marker="+", linewidth=2));

In [ ]:
# диаграмма рассеяния количества преступлений по дням недели (scatter_matrix)
scatter_matrix(area_weekday_plot, alpha=0.6, figsize=(17, 17), diagonal='kde');

In [ ]:
# распределение количества преступлений в любой день недели (например:пятница)
area_weekday_plot['Friday'].hist(density=True, color = '#bcbd') 
area_weekday_plot['Friday'].plot(kind='kde', figsize=(10,8), color = '#ffd300');

In [ ]:
# распределение количества преступлений в любой день недели (например:пятница)
%config InlineBackend.figure_format = 'retina' 
sns.distplot(area_weekday_plot['Friday'], color = '#bcbd', bins = 10);

In [ ]:
# гистограмма распределения количества преступлений в зависимости от района
area_occ = crime_data['Area Name'].value_counts()
area_occ.plot(kind = 'bar', figsize=(10,8), color = '#ffbd9d', rot = 45);

In [ ]:
# создание сводной таблицы для анализа по годам
area_year = crime_data.pivot_table(
                        index = 'Area Name', 
                        columns = crime_data['Date_Occurred_Full'].dt.strftime('%Y'), 
                        values = 'Date_Occurred_Full', 
                        aggfunc = 'count')

In [ ]:
# распределение количества преступлений за любой год (например: 2010 год)
sns.distplot(area_year['2010'], color = '#bcbd', bins = 10);

In [ ]:
# динамика изменений количества преступлений в сравнении 2010 и 2017 года (через jointplot)
sns.jointplot(x='2010', y='2017', data=area_year, kind='scatter', color = '#ddcaff', height = 5);

In [ ]:
# годовая динамика изменения количества преступлений по районам (линейный график с легендами и удобным масштабом)
area_year.T.plot(grid = True, figsize = (15, 6), colormap = 'Set2', xlim = (0,7));

In [ ]:
# создание сводной таблицы для анализа по преступлениям
crime_weekday = crime_data.pivot_table(
                        index = 'Weekday_Occurred', 
                        columns = 'Crime Code Description', 
                        values = 'Date_Occurred_Full', 
                        aggfunc = 'count')
crime_weekday

In [ ]:
# количество отдельно взятого вида преступления распределенного по дням недели
crime_weekday['BATTERY POLICE (SIMPLE)'].plot(kind = 'bar',  figsize = (8, 6), rot = 20, color = '#d3c2f4');

In [ ]:
# функция подсветки максимальных значений в таблице
def highlight_max(s):
    '''
    highlight the maximum in a Series red.
    '''
    is_max = s == s.max()
    return ['background-color: #ff6444' if v else '' for v in is_max]

In [ ]:
# функция подсветки минимальных значений в таблице
def highlight_min(s):
    '''
    highlight the minimum in a Series green.
    '''
    is_min = s == s.min()
    return ['background-color: #afff9d' if v else '' for v in is_min]

In [ ]:
# построение таблицы (вид преступления по дням недели) 
# подсветка максимальных и минимальных значений для каждого вида преступления разным цветом
crime_weekday.style.apply(highlight_max).apply(highlight_min)

In [ ]:
# создание датафрейма для анализа преступлений по неделям и районам
crime_weekday_area = crime_data[['Area Name', 'Crime Code Description', 'Weekday_Occurred']].value_counts().reset_index()
crime_weekday_area.rename(columns = {0: 'Crime_Occured'}, inplace = True)
crime_weekday_area_Central = crime_weekday_area[crime_weekday_area['Area Name'] == 'Central']
crime_weekday_area_Central['Area Name'] = crime_weekday_area_Central['Area Name'].astype(str)
crime_weekday_area_Central['Crime Code Description'] = crime_weekday_area_Central['Crime Code Description'].astype(str)

In [ ]:
# сравнение распределения 20 наиболее часто встречающихся преступлений 
# по дням недели для заданного района (показать через boxplot)
sns.boxplot(x="Weekday_Occurred", y="Crime_Occured", 
            data=crime_weekday_area_Central.head(20), palette = 'pastel');

In [ ]:
# анализ коэфициента корреляции вида преступления и дня недели 
# (результат в виде heatmap на которой отображена только корреляция вида преступления от дня недели)
# создание сводной таблицы для анализа корреляции
crime_weekday_area_pt = crime_weekday_area_Central.pivot_table(
                        index = 'Weekday_Occurred', 
                        columns = 'Crime Code Description', 
                        values = 'Crime_Occured', 
                        aggfunc = 'sum')
crime_weekday_area_pt.corr()

In [ ]:
sns.heatmap(crime_weekday_area_pt.corr().tail(1));

In [ ]:
# линейный график коэффициента корреляции дня недели и вида преступления в абсолютных значениях
# abs(crime_weekday_area_pt.corr()).plot(figsize = (15,5));

In [ ]:
# для удобства анализа преступлений по неделям и районам транспонирую таблицк из второго задания
area_weekday = area_weekday_plot.T

In [ ]:
# построение линейного графика для трех любых районов
trace0 = go.Scatter(
    x = area_weekday.index,
    y = area_weekday['Central'],
    name = 'Central'
)

trace1 = go.Scatter(
    x = area_weekday.index,
    y = area_weekday['Southwest'],
    name='Southwest'
)

trace2 = go.Scatter(
    x = area_weekday.index,
    y = area_weekday['Southeast'],
    name='Southeast'
)

data = [trace0, trace1, trace2]
layout = {'title': 'Statistics of Crimes Occurred'}

fig = go.Figure(data=data, layout=layout)

iplot(fig, show_link=False)

In [ ]:
# построение линейного графика для всех районов сразу
data = []

for col in area_weekday.columns:
    data.append(
        go.Scatter(x = area_weekday.index,
                   y = area_weekday[col], 
                   name = col)
    )

layout = {'title': 'Statistics of Crimes Occurred'}    
    
fig = go.Figure(data=data, layout=layout) 

iplot(data, show_link = False)

In [ ]:
# копирование части датафрейма для расчёта доли преступлений
area_data = crime_data['Area Name'].value_counts().reset_index()
area_data.rename(columns = {'index': 'Area Name', 'Area Name': 'Crime_Occurred'}, inplace = True)
area_data['Crime_Occurred_Share'] = 100*area_data['Crime_Occurred']/area_data['Crime_Occurred'].sum()
area_data

In [ ]:
# Доля преступлений в конкретном районе от общего количества 
# Bar график для всех районов по всей статистике
trace0 = go.Bar(
    x=area_data['Area Name'],
    y=area_data.Crime_Occurred_Share,
    name='Crime Occurred Share',
    orientation = 'v'
)


data = [trace0]
layout = {'title': 'Crime Occurred Share'}

fig = go.Figure(data=data, layout=layout)

iplot(fig, show_link=False)

In [ ]:
# транспонирование сводной таблицы из 3 задания для рассчёта
# доли преступлений в конкретном районе от общего количества для всех районов по годам
area_year_share = area_year.T

In [ ]:
# Доля преступлений в конкретном районе от общего количества 
# Bar график для всех районов по годам
data = []

for col in area_year_share.columns:
    data.append(
        go.Bar(x = area_year_share.index,
                   y = 100*area_year_share[col]/area_year_share.sum(axis = 1), 
                   name = col)
    )

layout = {'title': 'Statistics of Crimes Occurred'}    
    
fig = go.Figure(data=data, layout=layout) 

iplot(data, show_link = False)

In [ ]:
# box график статистики всех преступлений в распределении по годам
data = []

for i in area_year_share.index:
    data.append(
        go.Box(y = area_year_share.loc[i],
                   name = i)
    )

iplot(data, show_link = False)

In [ ]:
# количество преступлений от дня недели для отдельного района
traces = []
for area in area_weekday.columns:
    traces.append(
        go.Scatter(
            x=area_weekday[area].index,
            y=area_weekday[area].values,
            name = area, 
            visible = (area == '77th Street'))
    )
    
layout = go.Layout(
    title='Count of crimes for area by weekdays',
    updatemenus=list([
        dict(
            x=-0.05,
            y=1,
            yanchor='top',
            buttons=list([
                dict(
                    args=['visible', [True] + [False]*20],
                    label='77th Street',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False] + [True] + [False]*19],
                    label='Central',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*2 + [True] + [False]*18],
                    label='Devonshire',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*3 + [True] + [False]*17],
                    label='Foothill',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*4 + [True] + [False]*16],
                    label='Harbor',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*5 + [True] + [False]*15],
                    label='Hollenbeck',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*6 + [True] + [False]*14],
                    label='Mission',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*7 + [True] + [False]*13],
                    label='N Hollywood',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*8 + [True] + [False]*12],
                    label='Newton',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*9 + [True] + [False]*11],
                    label='Northeast',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*10 + [True] + [False]*10],
                    label='Olympic',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*11 + [True] + [False]*9],
                    label='Pacific',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*12 + [True] + [False]*8],
                    label='Rampart',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*13 + [True] + [False]*7],
                    label='Southeast',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*14 + [True] + [False]*6],
                    label='Southwest',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*15 + [True] + [False]*5],
                    label='Topanga',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*16 + [True] + [False]*4],
                    label='Van Nuys',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*17 + [True] + [False]*3],
                    label='West Valley',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*18 + [True] + [False]*2],
                    label='Hollywood',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*19 + [True] + [False]*1],
                    label='West LA',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*20 + [True]],
                    label='Wilshire',
                    method='restyle'
                )          
            ]),
        )
    ]),
)

fig = {'data': traces, 'layout': layout}
iplot(fig, show_link=False)

In [ ]:
area_weekday_t = area_weekday.T

In [ ]:
# количество преступлений от района для отдельного дня недели
traces = []
for weekday in area_weekday_t.columns:
    traces.append(
        go.Scatter(
            x=area_weekday_t[weekday].index,
            y=area_weekday_t[weekday].values,
            name = weekday, 
            visible = (weekday == 'Friday'))
    )
    
layout = go.Layout(
    title='Count of crimes for weekday by areas',
    updatemenus=list([
        dict(
            x=-0.05,
            y=1,
            yanchor='top',
            buttons=list([
                dict(
                    args=['visible', [True] + [False]*6],
                    label='Friday',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False] + [True] + [False]*5],
                    label='Monday',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*2 + [True] + [False]*4],
                    label='Saturday',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*3 + [True] + [False]*3],
                    label='Sunday',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*4 + [True] + [False]*2],
                    label='Thursday',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*5 + [True] + [False]],
                    label='Tuesday',
                    method='restyle'
                ),
                dict(
                    args=['visible', [False]*6 + [True]],
                    label='Wednesday',
                    method='restyle'
                )         
            ]),
        )
    ]),
)

fig = {'data': traces, 'layout': layout}
iplot(fig, show_link=False)